<a href="https://colab.research.google.com/github/Methun18/oibsip_taskno/blob/main/Cleaning_Data2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import libraries

In [ ]:
import pandas as pd
import numpy as np

### Load data

In [ ]:
df=pd.read_csv("/content/AB_NYC_2019.csv")

In [ ]:
df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


In [ ]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48895 entries, 0 to 48894
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              48895 non-null  int64  
 1   name                            48879 non-null  object 
 2   host_id                         48895 non-null  int64  
 3   host_name                       48874 non-null  object 
 4   neighbourhood_group             48895 non-null  object 
 5   neighbourhood                   48895 non-null  object 
 6   latitude                        48895 non-null  float64
 7   longitude                       48895 non-null  float64
 8   room_type                       48895 non-null  object 
 9   price                           48895 non-null  int64  
 10  minimum_nights                  48895 non-null  int64  
 11  number_of_reviews               48895 non-null  int64  
 12  last_review                     

# Handle Missing Data
Option 1: Fill Missing Data

In [ ]:
df['name'].fillna('Unknown', inplace=True)
df['host_name'].fillna('Unknown', inplace=True)

df['reviews_per_month'].fillna(0, inplace=True)

df['last_review'].fillna('1970-01-01', inplace=True)

df['last_review'] = pd.to_datetime(df['last_review'])


<ipython-input-5-ea0c02996ecf>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['name'].fillna('Unknown', inplace=True)
<ipython-input-5-ea0c02996ecf>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.me

Option 2: Drop Rows with Missing Data

In [ ]:
df.dropna(subset=['name', 'host_name'], inplace=True)

# Remove Duplicates

In [ ]:
df.drop_duplicates(inplace=True)

# Standardize Data

In [ ]:

df = df[df['price'] > 0]
df = df[df['availability_365'].between(0, 365)]
df.dtypes

,0
id,int64
name,object
host_id,int64
host_name,object
neighbourhood_group,object
neighbourhood,object
latitude,float64
longitude,float64
room_type,object
price,int64


# Detect and Handle Outliers
Method 1:Using Interquartile Range (IQR)

In [ ]:
def detect_outliers(col):
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[col] < lower_bound) | (df[col] > upper_bound)]


In [ ]:
outliers_price = detect_outliers('price')

In [ ]:
df = df[~df.isin(outliers_price)].dropna()

Method 2: Use Z-Score for Outlier Detection

In [ ]:
from scipy import stats
z_scores = np.abs(stats.zscore(df['minimum_nights']))
df = df[(z_scores < 3)]


# Verify Data Integrity

In [ ]:
df.isnull().sum()
df.info()
df.describe()


<class 'pandas.core.frame.DataFrame'>
Index: 45619 entries, 0 to 48894
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   id                              45619 non-null  float64       
 1   name                            45619 non-null  object        
 2   host_id                         45619 non-null  float64       
 3   host_name                       45619 non-null  object        
 4   neighbourhood_group             45619 non-null  object        
 5   neighbourhood                   45619 non-null  object        
 6   latitude                        45619 non-null  float64       
 7   longitude                       45619 non-null  float64       
 8   room_type                       45619 non-null  object        
 9   price                           45619 non-null  float64       
 10  minimum_nights                  45619 non-null  float64       
 11  number_

,id,host_id,latitude,longitude,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
count,4.561900e+04,4.561900e+04,45619.000000,45619.000000,45619.000000,45619.000000,45619.000000,45619,45619.000000,45619.000000,45619.000000
mean,1.891529e+07,6.645477e+07,40.728430,-73.950655,119.997896,5.869725,24.025932,2009-04-09 17:49:15.584295936,1.113216,6.631974,108.863960
min,2.539000e+03,2.438000e+03,40.499790,-74.244420,10.000000,1.000000,0.000000,1970-01-01 00:00:00,0.000000,1.000000,0.000000
25%,9.452808e+06,7.780024e+06,40.689190,-73.981870,65.000000,1.000000,1.000000,2016-06-12 00:00:00,0.050000,1.000000,0.000000
50%,1.954251e+07,3.038419e+07,40.721740,-73.954340,100.000000,2.000000,6.000000,2019-01-08 00:00:00,0.390000,1.000000,38.000000
75%,2.891583e+07,1.056405e+08,40.763380,-73.934245,159.000000,5.000000,25.000000,2019-06-20 00:00:00,1.630000,2.000000,215.000000
max,3.648724e+07,2.743213e+08,40.913060,-73.712990,334.000000,65.000000,629.000000,2019-07-08 00:00:00,58.500000,327.000000,365.000000
std,1.091388e+07,7.761964e+07,0.055325,0.046531,68.128673,8.886530,45.379043,NaN,1.615635,30.998982,130.069682


# Save Cleaned Data

In [ ]:
df.to_csv('cleaned_dataset.csv', index=False)
df

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539.0,Clean & quiet apt home by the park,2787.0,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149.0,1.0,9.0,2018-10-19,0.21,6.0,365.0
1,2595.0,Skylit Midtown Castle,2845.0,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225.0,1.0,45.0,2019-05-21,0.38,2.0,355.0
2,3647.0,THE VILLAGE OF HARLEM....NEW YORK !,4632.0,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150.0,3.0,0.0,1970-01-01,0.00,1.0,365.0
3,3831.0,Cozy Entire Floor of Brownstone,4869.0,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89.0,1.0,270.0,2019-07-05,4.64,1.0,194.0
4,5022.0,Entire Apt: Spacious Studio/Loft by central park,7192.0,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80.0,10.0,9.0,2018-11-19,0.10,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48890,36484665.0,Charming one bedroom - newly renovated rowhouse,8232441.0,Sabrina,Brooklyn,Bedford-Stuyvesant,40.67853,-73.94995,Private room,70.0,2.0,0.0,1970-01-01,0.00,2.0,9.0
48891,36485057.0,Affordable room in Bushwick/East Williamsburg,6570630.0,Marisol,Brooklyn,Bushwick,40.70184,-73.93317,Private room,40.0,4.0,0.0,1970-01-01,0.00,2.0,36.0
48892,36485431.0,Sunny Studio at Historical Neighborhood,23492952.0,Ilgar & Aysel,Manhattan,Harlem,40.81475,-73.94867,Entire home/apt,115.0,10.0,0.0,1970-01-01,0.00,1.0,27.0
48893,36485609.0,43rd St. Time Square-cozy single bed,30985759.0,Taz,Manhattan,Hell's Kitchen,40.75751,-73.99112,Shared room,55.0,1.0,0.0,1970-01-01,0.00,6.0,2.0
